In [ ]:
import pandas as pd
import numpy as np
import feather
from sklearn import preprocessing as pe
from tqdm import tqdm

In [ ]:
df_train = feather.read_dataframe('../cache/train_stage2.feather')

In [ ]:
df_train.head()

In [ ]:
df_test = feather.read_dataframe('../cache/test_stage2.feather')

In [ ]:
df_test.head()

In [ ]:
y = df_train['Class'].values
df_train = df_train.drop(['Class'], axis=1)

test_ids = df_test['ID'].values

In [ ]:
df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True)

In [ ]:
df_all.head()

In [ ]:
df_all.shape

In [ ]:
df_all['GeneVar'] = df_all['Gene'] + ' ' + df_all['Variation']

In [ ]:
df_all['Gene_Share'] = df_all.apply(lambda row: sum([1 for w in row['Gene'].split(' ') 
                                                   if w in row['Text'].split(' ')]), axis=1)



In [ ]:
df_all['Variation_Share'] = df_all.apply(lambda row: sum([1 for w in row['Variation'].split(' ') 
                                                        if w in row['Text'].split(' ')]), axis=1)

In [ ]:
df_all['Gene_Variation_Share'] = df_all.apply(lambda row: sum([1 for w in row['GeneVar'].split(' ') 
                                                        if w in row['Text'].split(' ')]), axis=1)

In [ ]:
df_all.head()

In [ ]:
df_all['GL'] = df_all['Gene'].apply(lambda x: len(x))

In [ ]:
sorted(set(df_all['GL'].values))

In [ ]:
df_all['VL'] = df_all['Variation'].apply(lambda x: len(x))

In [ ]:
sorted(set(df_all['VL'].values))

In [ ]:
max_len=55

In [ ]:
for i in range(max_len+1):
    df_all['Gene_'+str(i)] = df_all['Gene'].map(lambda x: str(x[i]) if len(x)>i else '')
    df_all['Variation_'+str(i)] = df_all['Variation'].map(lambda x: str(x[i]) if len(x)>i else '')
    df_all['GeneVar_'+str(i)] = df_all['GeneVar'].map(lambda x: str(x[i]) if len(x)>i else '')

In [ ]:
df_all.head()

In [ ]:
gene_var_lst = sorted(list(df_train.Gene.unique()) + list(df_train.Variation.unique()))

gene_var_lst = [x for x in gene_var_lst if len(x.split(' '))==1]

i_ = 0

for el in tqdm(gene_var_lst):
    df_all['GV_'+str(el)] = df_all['Text'].map(lambda x: str(x).count(str(el)))
    i_ += 1

In [ ]:
df_all.head()

In [ ]:
s = df_all['Gene'].value_counts()

In [ ]:
print(s['BRCA1'])

In [ ]:
df_all['G_VC'] = df_all['Gene'].apply(lambda gene: s[str(gene)])

In [ ]:
df_all.G_VC[df_all.Gene == 'BRCA1'].count()

In [ ]:
s = df_all['Variation'].value_counts()

In [ ]:
df_all['V_VC'] = df_all['Variation'].apply(lambda var: s[str(var)])

In [ ]:
s = df_all['GeneVar'].value_counts()

In [ ]:
df_all['GV_VC'] = df_all['GeneVar'].apply(lambda var: s[str(var)])

In [ ]:
D = 2 ** 24

In [ ]:
df_all['hash'] = df_all['Gene'] + '_' + df_all['Variation'] + '_' + df_all['Text']
                

In [ ]:
df_all.head()

In [ ]:
df_all['hash'] = df_all['hash'].apply(lambda x: hash(x) % D)

In [ ]:
df_all.head()

In [ ]:
df_all.shape

In [ ]:
for c in tqdm(df_all.columns):
    if df_all[c].dtype == 'object':
        if c in ['Gene','Variation','GeneVar']:
            lbl = pe.LabelEncoder()
            df_all[c+'_lbl_enc'] = lbl.fit_transform(df_all[c].values)  
            df_all[c+'_len'] = df_all[c].map(lambda x: len(str(x)))
            df_all[c+'_words'] = df_all[c].map(lambda x: len(str(x).split(' ')))
        elif c != 'Text':
            lbl = pe.LabelEncoder()
            df_all[c] = lbl.fit_transform(df_all[c].values)
        if c=='Text': 
            df_all[c+'_len'] = df_all[c].map(lambda x: len(str(x)))
            df_all[c+'_words'] = df_all[c].map(lambda x: len(str(x).split(' '))) 



In [ ]:
df_train = df_all.iloc[:len(df_train)]
df_test = df_all.iloc[len(df_train):]

In [ ]:
df_train['Class'] = y

In [ ]:
df_train.shape

In [ ]:
df_test.shape

In [ ]:
df_train.head()

In [ ]:
df_train['hash'].value_counts()

In [ ]:
df_test.head()

In [ ]:
feather.write_dataframe(df_train, '../cache/train_stage2_fe.feather')

In [ ]:
feather.write_dataframe(df_test, '../cache/test_stage2_fe.feather')